<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%202/NLP_07_extraccio%CC%81n_de_caracteri%CC%81sticas__Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de características en grandes volúmenes de datos
Para extraer las características de un corpus muy grande de textos, conviene procesarlo mediante objetos *stream*. Así no se carga todo el corpus en memoria sino que se analiza documento a documento.

## Conjunto de textos de "mundocine"
Vamos a utilizar un conjunto de datos formado por una serie de críticas de películas de cine, almacenadas en formato XML en el directorio `'criticas'` (una crítica por archivo). Hemos preparado una función de tipo `generator` que procesa el directorio donde están los archivos de las críticas y devuelve por cada archivo XML una tupla con 4 valores: 
 - Nombre de la película (*string*)
 - Resumen breve de la crítica (*string*)
 - Texto de la crítica (*string*)
 - Valoración de la película (*int* de 1 a 5)

In [ ]:
import os, re
from xml.dom.minidom import parseString

def parse_folder(path):
    """generator that reads the contents of XML files in a folder
    Returns the <body> of the <review> in each XML file.
    XML files encoded as 'latin-1'"""
    for file in sorted([f for f in os.listdir(path) if f.endswith('.xml')],
                        key=lambda x: int(re.match(r'\d+',x).group())):
        with open(os.path.join(path, file), encoding='latin-1') as f:
            doc=parseString(re.sub(r'(<>)|&|(<-)', '', f.read()))

            titulo = doc.documentElement.attributes["title"].value

            btxt = ""
            review_bod = doc.getElementsByTagName("body")
            if len(review_bod) > 0:
                for node in review_bod[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        btxt += node.data + " "

            rtxt = ""
            review_summ = doc.getElementsByTagName("summary")
            if len(review_summ) > 0:
                for node in review_summ[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        rtxt += node.data + " "
                        
            rank = int(doc.documentElement.attributes["rank"].value)
            
            yield titulo, rtxt, btxt, rank


Para entender lo que hace **yield**, dejo por aquí este enlace donde se explica.

Así por encima, yield es útil para crear un generador y no cargar todos los datos a la vez. Únicamente cargará un elemento, y actuará como un iterador, tal y como se puede apreciar en el ejercico 1.

Para una explicación mejor, consultar este link:
https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do

### Ejercicio 1
Carga la primera crítica del directorio usando el método `next` sobre la función `parse_folder` en el objeto `critica`. Muestra sus 4 valores.

In [ ]:
critica = parse_folder('criticas')

In [ ]:
next(critica)

('Los Increibles',
 '¿Por qué las peliculas animadas son mucho mejores que las normales? ',
 'Es que no la cagan en ninguna, todas las pelis que hacen son cada vez mejores. Incluso no caen en los estereotipos en los que es tan fácil caer, que es repetirse y repetirse, como la mierda de American Pie 3, que es volver a hacer los mismos chistes, mal contados y previsibles a la milla. ¿Qué se esconde detrás de esas peliculas? ¿Cómo es que no te aburren ni un solo momento? ¿Cómo es que entretienen por igual a niños que a mayores? No tengo ni puta idea, lanzo las preguntas al aire, si alguien las sabe, que las responda ahora mismo en alto en su casa. Esta vez me ha pasado algo que no me habia pasado nunca antes con una pelicula (bueno si, con Pocahontas un poquito). Y es que la madre y la niña me han puesto brutito. La madre al comienzo de la pelicula esta quesito, ella toda mona, y luego al final de la peli cuando se pone en plan luchadora tambien, en cambio cuando esta de madre es el bajon

In [ ]:
#es un iterador:

next(critica)

('La casa de cera',
 'Casquería, terror y cera a partes iguales ',
 'Tras una insufrible primera media hora, la casa de cera se empieza a calentar, nunca mejor dicho, y nos empieza a "deleitar" con el gore terror más asqueroso que he visto hace tiempo. Es de risa el comienzo de la peli, con unos personajes más planos que el escáner cerebral de Mariah Carey y una chispa americana que ríete tu de Juanito Navarro. La casa de cera comienza a tomar forma a los 40 minutos más o menos cuando a una de las rubias tontas del grupo se le ocurre seguir el rastro del mal olor que hay en el bosque. El comienzo se torna grotesco y surrealista cuando después de sacarla de un lodazal de sangre de la que ella misma podría haber salido perfectamente, los chicos encuentran una mano sepultada entre la sangraza y las pieles. Entonces hace aparición en la peli el tercer hermano matarife con el cheroki tuneao de mierda hasta la bandera, un papel que las mentes de pensamiento bineuronal no tardarán en identifi

In [ ]:
#afecta también a los for. Comenzará desde el tercer elemento en este for, y no desde el primero,
# ya que es afectado por los dos next(critica) hechos anteriormente
for c in critica:
    print(c)
    break

('Spiderman 2', 'Poniendo a parir una peli que no cuenta nada ', 'Es un dolor esto del cine. Yo ya voy con miedo. Primero por las colas tan largas, segundo por ver quien te toca al lado, y tercero por los guiones cada vez mas gilipollescos que se curran. El cine esta cayendo en picado, como la tele, como el mundo editorial, donde cualquier gualtrapa que por solo salir en la tele te "escribe" un "libro". Lo de Spiderman es para coger al director y darle cuatro ostias. Pero a cuento de que te mete tanta filosofia de cura en boca de tantos personajes de la peli?, nos quieren dejar vivir un poco?. Yo voy al cine a ver a Spiderman como zurra a los malos, no a enmierdarme en su vida privada y sus problemas psicologicos. Me la suda la novia de Spiderman, esa lerda con cara de pastora de los montes de Usurbil que parece que no folla nunca, y que es una pretenciosa que le dice al final que le va a salvar la vida, y que le monta un pollo porque no ha podido ir a ver su mierda de obra de teatro. 

## Extracción de características básicas
Utilizando la librería `textaCy` procesamos cada documento y generamos una serie de estadísticos. \
Guardaremos los resultados en un objeto `DataFrame` de Pandas.\
Como característica de cada crítica vamos a extraer:
- Título de la película
- Longitud (en caracteres) del resumen
- Longitud (en caracteres) del texto de la crítica
- Puntuación de la crítica

### Ejercicio 2
Completa el código siguiente para generar el `DataFrame`

In [ ]:
import pandas as pd

#creamos una lista en blanco
datos = []

#recorremos las críticas y calculamos sus métricas
for c in parse_folder("criticas"):
    datos.append({
        'título': c[0],
        'LongResumen': len(c[1]),
        'LongCritica': len(c[2]),
        'puntuación': c[3]
    })

resumen = pd.DataFrame(datos)

In [ ]:
resumen

,título,LongResumen,LongCritica,puntuación
0,La guerra de los mundos,32,2951,1
1,Stars Wars III La venganza de los Sith,30,3954,3
2,Los Increibles,68,2467,5
3,Spiderman 2,45,3403,2
4,La casa de cera,42,1696,2
5,El internado,31,1163,2
6,La llave del mal,36,1781,3
7,May,28,3343,4
8,Cinderella Man,33,2885,2
9,La Novia Cadaver,40,736,3


## Limpieza de texto
Para poder extraer las características vectoriales del corpus es conveniente realizar primero una limpieza y pre-procedado de cada documento.\
Vamos a suponer que queremos preparar este conjunto de textos para entrenar un modelo que prediga la puntuación de cada crítica a partir del texto de la crítica.\
Realizaremos el siguiente procesado:
- Introducir un espacio después de determinados signos de puntuación (".", "?") para que el tokenizado sea correcto
- Separar el texto en *tokens*
- Eliminar los *tokens* de tipo *stop-word*, signos de puntuación o espacios o de longitud 1
- Convertir las entidades de tipo `PER` al token *persona*
- Lematizar el texto y pasarlo a minúsculas

In [ ]:
import spacy

nlp = spacy.load("es_core_news_sm")

def normaliza(texto):
    #separamos después de ciertos signos de puntuación
    texto = re.sub(r"([\.\?])", r"\1 ", texto)
    doc = nlp(texto)
    tokens = [t for t in doc if not t.is_punct and not t.is_stop and not t.is_space and len(t.text)>1]
    palabras = []
    for t in tokens:
        if t.ent_iob_=='B' and t.ent_type_=='PER':
            palabras.append('persona')
        elif t.ent_iob_=='I' and t.ent_type_=='PER':
            continue
        else:
            palabras.append(t.lemma_.lower())
    salida = ' '.join(palabras)
    
    return salida

### Ejercicio 3
Comprueba su funcionamiento en la crítica previamente descargada (variable `critica`)

In [ ]:
criticas = parse_folder('criticas')
critica = next(criticas)

In [ ]:
normaliza(critica[2])

'gustar cine masa pelicula ver mundo parecer coñazo insufribl porquar prota tonto culo suerte peli lograr vencer convertir listo chorrada comienzo pelicula esfumar arte magia volver maduro inteligente peli persona huir huir dar tiro cabron meter par actor echarl comer aparte niña viejo metida cuerpo niña ver él hablar version original dar él freak cine creeran gracia nena puta madre causar pavor cria persona maduro horroroso niño niño ver él rol asusta hijo adolescente cruise subnormal dar él bofetada ver hueso mano fiel reflejo denominar manipulacion militar chico matar bicho ningun arma hale loco pensar venir saco quereis decir fanatismo locura alguien querer luchar medio muerte seguro jodar sobremanerir pelicula aparecer mongo abuelo familia salir dio viejo aparecer traje domingo maquillada sinsorgadar faltar persona volver exmujer mundo tranquilos contar peli pasar peli acabar restar misterio bodrio persona paso convertir director efecto especial contar historia disteis norteameric

### Análisis morfológico
En una crítica tiene mucha importancia los adjetivos utilizados.\
Crea una función para filtrar sólo los adjetivos utilizados en cada crítica (utiliza el lema de cada adjetivo).

In [ ]:
def extraer_adj(texto):
    texto = re.sub(r"([\.\?])", r"\1 ", texto)
    doc = nlp(texto)
    tokens = [t.lemma_ for t in doc if t.pos_=="ADJ"]
    
    return ' '.join(tokens)

Comprobamos su funcionamiento en la crítica previamente descargada (variable `critica`)

In [ ]:
extraer_adj(critica[2])

'insufribl maduro inteligente solo viejo original claro grande ver maduro horroroso asusta adolescente subnormal militar hale loco seguro viejo mejor igual solo bueno especial dificil peli estario aceptable'

## Extracción de características *sparse* con `scikit-learn`
Vamos a calculas las matrices de características *bag-of-words* y *tfidf* del conjunto de textos anterior.\
Vamos a usar la librería `scikit-learn` para vectorizar los documentos.

In [ ]:
#Para no tener que cargar todas las críticas en memoria,
#creamos un generador que devuelve iterativamente el
#texto procesado de cada crítica

def generaCritica(criticas):
    """Función de tipo generator que devuelve el
    texto normalizado de cada crítica.
    Entrada:
    criticas: objeto 'parse_folder' que itera
    sobre el directio de las críticas
    Salida:
    texto normalizado de cada crítica"""
    for c in criticas:
        yield normaliza(c[2])

Comprobamos su funcionamiento generando el texto normalizado de la primera crítica

In [ ]:
next(generaCritica(parse_folder("criticas")))

'gustar cine masa pelicula ver mundo parecer coñazo insufribl porquar prota tonto culo suerte peli lograr vencer convertir listo chorrada comienzo pelicula esfumar arte magia volver maduro inteligente peli persona huir huir dar tiro cabron meter par actor echarl comer aparte niña viejo metida cuerpo niña ver él hablar version original dar él freak cine creeran gracia nena puta madre causar pavor cria persona maduro horroroso niño niño ver él rol asusta hijo adolescente cruise subnormal dar él bofetada ver hueso mano fiel reflejo denominar manipulacion militar chico matar bicho ningun arma hale loco pensar venir saco quereis decir fanatismo locura alguien querer luchar medio muerte seguro jodar sobremanerir pelicula aparecer mongo abuelo familia salir dio viejo aparecer traje domingo maquillada sinsorgadar faltar persona volver exmujer mundo tranquilos contar peli pasar peli acabar restar misterio bodrio persona paso convertir director efecto especial contar historia disteis norteameric

Vectorizamos todo el conjunto de datos usando las funciones de `scikit-learn`.\
Estas funciones admiten un objeto `generator` como argumento de entrada.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

criticas = generaCritica(parse_folder("criticas"))
BoW_criticas = vect.fit_transform(criticas)
BoW_criticas

<29x2549 sparse matrix of type '<class 'numpy.int64'>'
	with 4129 stored elements in Compressed Sparse Row format>

In [ ]:
criticas = generaCritica(parse_folder('criticas'))
vect.fit(criticas)
BoW_criticas = vect.transform(criticas)
BoW_criticas

<0x2549 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
BoW_criticas = vect.transform(generaCritica(parse_folder('criticas')))
BoW_criticas

<29x2549 sparse matrix of type '<class 'numpy.int64'>'
	with 4129 stored elements in Compressed Sparse Row format>

### Ejercicio 4
Genera distintas variantes de matrices de características para el conjunto de las críticas. Prueba con:
- Matriz TF-IDF
- Matriz BoW con unigramas y bigramas
- Matriz TF-IDF eliminando las palabras menos frecuentes y las más frecuentes (mínimo de 2 y máximo de 5 documentos)
- Muestra cuáles son las palabras más frecuentes eliminadas (*atributo stop_words_ del vectorizador*)

In [ ]:
#matriz TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()

TFIDF_criticas = vect.fit_transform(generaCritica(parse_folder('criticas')))
TFIDF_criticas

<29x2549 sparse matrix of type '<class 'numpy.float64'>'
	with 4129 stored elements in Compressed Sparse Row format>

In [ ]:
#BoW de unigramas y bigramas
vect = CountVectorizer(ngram_range=(1,2))

BoW_criticas = vect.fit_transform(generaCritica(parse_folder('criticas')))
BoW_criticas

<29x7567 sparse matrix of type '<class 'numpy.int64'>'
	with 9253 stored elements in Compressed Sparse Row format>

En el caso de BoW, el tamaño es mayor que el TF-IDF ya que estamos teniendo en cuenta tanto unigramas (conjuntos de 1 palabra) y bigramas (conjuntos de 2 palabras)

In [ ]:
#Matriz TF-IDF eliminando las palabras menos frecuentes y las más frecuentes (mínimo de 2 y máximo de 5 documentos)

vect = TfidfVectorizer(min_df=2, max_df=5)

TFIDF_criticas = vect.fit_transform(generaCritica(parse_folder('criticas')))
TFIDF_criticas

<29x566 sparse matrix of type '<class 'numpy.float64'>'
	with 1523 stored elements in Compressed Sparse Row format>

In [ ]:
#Palabras más frecuencias eliminadas (atributo 'stop_words_' del vectorizador)

vect.stop_words_

{'cocaina',
 'laureado',
 'descarga',
 'escena',
 'medicina',
 'aliviar',
 'afincado',
 'digno',
 'maravillosa',
 'esfumar',
 'sobrepasar',
 'erotico',
 'asimilar',
 'crimen',
 'talante',
 'regreso',
 'curso',
 'unico',
 'radical',
 'destructiva',
 'mckee',
 'espós',
 'localización',
 'caserón',
 'pallo',
 'dejarir',
 'tecnico',
 'superávit',
 'heredero',
 'planteamiento',
 'calibrar',
 'recreado',
 'mejora',
 'geisha',
 'hecho',
 'apoderar',
 'serkis',
 'retomar',
 'mago',
 'retórico',
 'post',
 'esdla',
 'fallecido',
 'inconveniente',
 'mínimo',
 'mater',
 'kôji',
 'acertado',
 'llevado',
 'suegro',
 'personaje',
 'divertir',
 'actor',
 'loca',
 'intranqulidad',
 'superación',
 'globos',
 'escondiéndome',
 'contraterrorismo',
 'setenta',
 'omnipresente',
 'espantoso',
 'confundir',
 'anti',
 'nacionalista',
 'match',
 'irritar',
 'exageración',
 'descrito',
 'originalidad',
 'cercano',
 'significar',
 'técnica',
 'anakin',
 'veiamos',
 'tranquilizar',
 'luchador',
 'cruise',
 'casco'

In [ ]:
len(vect.stop_words_)

1983

## Extracción de Word embeddings
Ahora vamos a calcular los *word vectors* de las palabras de nuestro conjunto de datos, usando la clase `word2vec` de la librería `gensim`.\
Esta librería acepta como argumento de entrada un objeto `iterador` que generará el texto pre-procesado de la siguiente crítica en la secuencia.\
Vamos a usar las funciones de pre-procesado de la librería `gensim`.\
Primero definimos un objeto de tipo `streamed iterable` para recorrer las críticas. Se diferencia de un simple `generator` en que cada vez que se llama, se puede reiniciar la generación de la secuencia (necesario para el modelo `word2vec`)

In [ ]:
from gensim.utils import simple_preprocess
        
class PreprocesaCriticas(object):
    """Pre-procesa el corpus de críticas con la función 'simple_preprocess'
    de la librería gensim
    Entrada: directorio de críticas
    Salida: iterador sobre las críticas (como lista de tokens)"""
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for c in parse_folder(self.dirname):
            yield normaliza(c[2]).split(' ')

In [ ]:
#instanciamos un objeto para nuestro directorio
criticas = PreprocesaCriticas("criticas")

Para probar su funcionamiento con la primera crítica usamos el método `next` convirtiéndolo en *iterable*

In [ ]:
criticas_iter = iter(criticas)
print(next(criticas_iter))

['gustar', 'cine', 'masa', 'pelicula', 'ver', 'mundo', 'parecer', 'coñazo', 'insufribl', 'porquar', 'prota', 'tonto', 'culo', 'suerte', 'peli', 'lograr', 'vencer', 'convertir', 'listo', 'chorrada', 'comienzo', 'pelicula', 'esfumar', 'arte', 'magia', 'volver', 'maduro', 'inteligente', 'peli', 'persona', 'huir', 'huir', 'dar', 'tiro', 'cabron', 'meter', 'par', 'actor', 'echarl', 'comer', 'aparte', 'niña', 'viejo', 'metida', 'cuerpo', 'niña', 'ver', 'él', 'hablar', 'version', 'original', 'dar', 'él', 'freak', 'cine', 'creeran', 'gracia', 'nena', 'puta', 'madre', 'causar', 'pavor', 'cria', 'persona', 'maduro', 'horroroso', 'niño', 'niño', 'ver', 'él', 'rol', 'asusta', 'hijo', 'adolescente', 'cruise', 'subnormal', 'dar', 'él', 'bofetada', 'ver', 'hueso', 'mano', 'fiel', 'reflejo', 'denominar', 'manipulacion', 'militar', 'chico', 'matar', 'bicho', 'ningun', 'arma', 'hale', 'loco', 'pensar', 'venir', 'saco', 'quereis', 'decir', 'fanatismo', 'locura', 'alguien', 'querer', 'luchar', 'medio', 'm

Al contrario que el objeto generado con la función `generaCriticas` el objeto de `PreprocesaCriticas` se reinicia cada vez que lo iteramos

In [ ]:
criticas = PreprocesaCriticas("criticas")
for c in criticas:
    print(c)
    break

['gustar', 'cine', 'masa', 'pelicula', 'ver', 'mundo', 'parecer', 'coñazo', 'insufribl', 'porquar', 'prota', 'tonto', 'culo', 'suerte', 'peli', 'lograr', 'vencer', 'convertir', 'listo', 'chorrada', 'comienzo', 'pelicula', 'esfumar', 'arte', 'magia', 'volver', 'maduro', 'inteligente', 'peli', 'persona', 'huir', 'huir', 'dar', 'tiro', 'cabron', 'meter', 'par', 'actor', 'echarl', 'comer', 'aparte', 'niña', 'viejo', 'metida', 'cuerpo', 'niña', 'ver', 'él', 'hablar', 'version', 'original', 'dar', 'él', 'freak', 'cine', 'creeran', 'gracia', 'nena', 'puta', 'madre', 'causar', 'pavor', 'cria', 'persona', 'maduro', 'horroroso', 'niño', 'niño', 'ver', 'él', 'rol', 'asusta', 'hijo', 'adolescente', 'cruise', 'subnormal', 'dar', 'él', 'bofetada', 'ver', 'hueso', 'mano', 'fiel', 'reflejo', 'denominar', 'manipulacion', 'militar', 'chico', 'matar', 'bicho', 'ningun', 'arma', 'hale', 'loco', 'pensar', 'venir', 'saco', 'quereis', 'decir', 'fanatismo', 'locura', 'alguien', 'querer', 'luchar', 'medio', 'm

In [ ]:
for c in criticas:
    print(c)
    break

['gustar', 'cine', 'masa', 'pelicula', 'ver', 'mundo', 'parecer', 'coñazo', 'insufribl', 'porquar', 'prota', 'tonto', 'culo', 'suerte', 'peli', 'lograr', 'vencer', 'convertir', 'listo', 'chorrada', 'comienzo', 'pelicula', 'esfumar', 'arte', 'magia', 'volver', 'maduro', 'inteligente', 'peli', 'persona', 'huir', 'huir', 'dar', 'tiro', 'cabron', 'meter', 'par', 'actor', 'echarl', 'comer', 'aparte', 'niña', 'viejo', 'metida', 'cuerpo', 'niña', 'ver', 'él', 'hablar', 'version', 'original', 'dar', 'él', 'freak', 'cine', 'creeran', 'gracia', 'nena', 'puta', 'madre', 'causar', 'pavor', 'cria', 'persona', 'maduro', 'horroroso', 'niño', 'niño', 'ver', 'él', 'rol', 'asusta', 'hijo', 'adolescente', 'cruise', 'subnormal', 'dar', 'él', 'bofetada', 'ver', 'hueso', 'mano', 'fiel', 'reflejo', 'denominar', 'manipulacion', 'militar', 'chico', 'matar', 'bicho', 'ningun', 'arma', 'hale', 'loco', 'pensar', 'venir', 'saco', 'quereis', 'decir', 'fanatismo', 'locura', 'alguien', 'querer', 'luchar', 'medio', 'm

In [ ]:
criticas = generaCritica(parse_folder("criticas"))
for c in criticas:
    print(c)
    break

gustar cine masa pelicula ver mundo parecer coñazo insufribl porquar prota tonto culo suerte peli lograr vencer convertir listo chorrada comienzo pelicula esfumar arte magia volver maduro inteligente peli persona huir huir dar tiro cabron meter par actor echarl comer aparte niña viejo metida cuerpo niña ver él hablar version original dar él freak cine creeran gracia nena puta madre causar pavor cria persona maduro horroroso niño niño ver él rol asusta hijo adolescente cruise subnormal dar él bofetada ver hueso mano fiel reflejo denominar manipulacion militar chico matar bicho ningun arma hale loco pensar venir saco quereis decir fanatismo locura alguien querer luchar medio muerte seguro jodar sobremanerir pelicula aparecer mongo abuelo familia salir dio viejo aparecer traje domingo maquillada sinsorgadar faltar persona volver exmujer mundo tranquilos contar peli pasar peli acabar restar misterio bodrio persona paso convertir director efecto especial contar historia disteis norteamerica

In [ ]:
for c in criticas:
    print(c)
    break

sioux decia amigo trilogia pelicula llegar entrar consideración malo suda medio peli momento momento soporífero aguantar memo charla nidito amor tochazo rollo meter politico batallita molar sobresaliente escasez mental retarded anakin pelicula anterior aparecer tio listo ver liar palpatine burda palabrerio chantajeadoro quedar tonto ver anaquino venir persona quino joaquin sueño persona pasar obscuro escribiar encontrarar remedio calle preguntar tia ver soñar malo decir él solucion sueño dejar meterl puntita tolai caer juego persona caer mir él transalpinobueno hablar persona emperador malo maloso liante viejo habeis percatado rollo recordar extrañamente persona accion irak matar traera paz enemigo enredar gente cós malo hacer él armada clón quitar sombrero semejante telaraña montado tomar mr bush irak jedaisharta gracia jedai jedi vida retorno jedi deciar niño alguien decir retorno jedai dar toña marica hablar anterioridad ser comen follar alquien ver follar jedi comer preguntar coño 

Calculamos los vectores de palabras de todo el corpus con el modelo `word2vec` que acepta como argumento de entrada un objeto de tipo `iterator` como el creado

In [ ]:
#Cálculo de los vectores de palabras
from gensim.models import Word2Vec

criticas = PreprocesaCriticas('criticas')

model = Word2Vec(criticas, #iterador con los documentos
                               size=10,          #tamaño del vector
                               window=5,         #nº de términos adyacentes que usamos para el cálculo
                               min_count=5,      #nº mínimo de apariciones del término para contarlo
                               iter=100
                              )

#una vez entrenado el modelo nos quedamos con los vectores calculados
#si no se van a actualizar los vectores con nuevos documentos
model = model.wv
len(model.vocab)

189

Seleccinamos aleatoriamente 25 palabras del conjunto calculado

In [ ]:
import numpy as np

palabras_sample = np.random.choice(model.index2word, 25, replace=False)

In [ ]:
palabras_sample

array(['único', 'representar', 'hora', 'yo', 'poner', 'madre', 'terror',
       'lleno', 'original', 'principal', 'trama', 'problema', 'cara',
       'contar', 'pantalla', 'tele', 'director', 'actor', 'escribir',
       'muerte', 'tipo', 'demostrar', 'saber', 'venir', 'niña'],
      dtype='<U13')

In [ ]:
model['desarrollar']

KeyError: "word 'desarrollar' not in vocabulary"

### Ejercicio 5
Comprueba cómo funciona el modelo buscando las palabras más similares semánticamente a "trama", "peli" y "película"

In [ ]:
model.similarity('peli', 'pelicula')

0.7882221

In [ ]:
model.similarity('pelicula', 'película')

0.17501305

EL resultado de arriba es directamente un **problema en el preprocesado**.

Al no haber eliminado los acentos en el preprocesado (o no haber corregido las palabras correctamente), tenemos dos entradas distintas para película: con acento y sin acento.

No sólo eso, sino que el modelo ha encontrado una similaridad baja para esas palabras, por lo que podría estar confunidendo al modelo

## Extracción de características *sparse* con `gensim`
Vamos a generar las matrices TF-IDF y BoW del corpus con la librería Gensim mediante *data streming* para no tener que cargar en memoria los textos.\
No podemos usar simplemente un generador con el *corpus* para crear el modelo BoW/TF-IDF porque necesitamos re-iniciar la iteración sobre los textos después de crear el diccionario. Por tanto creamos una clase sobre el generador del *corpus* que genere un *streamed iterable*

*Este conjunto de códigos está hecho de prueba por el profesor para ir viendo y probando*

In [ ]:
import gensim

diccionario = gensim.corpora.Dictionary(criticas)

In [ ]:
len(diccionario.token2id)

2550

In [ ]:
bow = [diccionario.doc2bow(v) for v in criticas] #bag of words

In [ ]:
len(bow)

29

In [ ]:
print(bow[0]) #(id de la palabra, frecuencia de la palabra)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 3), (21, 1), (22, 2), (23, 1), (24, 1), (25, 2), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 2), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 2), (108, 2), (109, 1), (110, 1),

*Fin del conjunto de códigos de prueba*

*En estos códigos de arriba hemos cargado todo el bag of words a la vez.*

*La idea sería cargarlo de forma iterativa como se ha hecho antes (al principio del notebook) con cada crítica, y no tenerlo todo a la vez en memoria*

In [ ]:
import gensim

def iter_criticas(top_directory):
    """
    Generator: itera sobre todos los documentos del 
    directorio, devolviendo un documento cada vez,
    como lista de tokens.
    """
    for c in parse_folder(top_directory):
        yield normaliza(c[2]).split(' ')
 
class TxtSubdirsCorpus(object):
    """
    Iterable: en cada iteración devuelve el vector bag-of-words
    del siguiente documento (crítica).
    
    Procesa un documento cada vez usando un generator, así
    nunca carga el corpus entero en RAM.
    """
    def __init__(self, top_dir):
        self.top_dir = top_dir
        #crea el diccionario = mapeo de documentos a sparse vectors
        self.dictionary = gensim.corpora.Dictionary(iter_criticas(top_dir))
 
    def __iter__(self):
        """
        __iter__ es un generator => TxtSubdirsCorpus es un streamed iterable.
        """
        for tokens in iter_criticas(self.top_dir):
            # transforma cada doc (lista de tokens) en un vector sparse uno a uno
            yield self.dictionary.doc2bow(tokens)

Para crear el BoW del *corpus* simplemente recorremos el objeto `TxtSubdirsCorpus` instanciado para el directorio de las críticas

In [ ]:
corpus = TxtSubdirsCorpus('criticas')
 
# generamos el BoW
for vector in corpus:
    print(vector)
    break